### Hotelling T2法（多次元バージョン）
参考文献:<br>
> 井出剛. 2015. 入門機械学習による異常検知. コロナ社. pp.37-48.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from typing import Union, Literal

#### 多次元データにおける、ホテリングT2法の特徴
1次元の場合に加え以下が加わる
1. 多次元データの相関構造を考慮した異常検知が可能

#### ホテリングT2法（多次元）の手順 <br>
#### 1. 分布の推定
データは多次元正規分布に従うと仮定する。
$$
N(\mathbfit{x}| \boldsymbol{\mu}, \boldsymbol{\Sigma}) = \frac{|\boldsymbol{\Sigma}|^{-1/2}}{(2 \pi)^{M/2}} \exp \biggl\{ - \frac{1}{2} (\mathbfit{x} -  \boldsymbol{\mu})^\top \boldsymbol{\Sigma}^{-1} (\mathbfit{x} -  \boldsymbol{\mu}) \biggr\}
$$

上記において、$M$ は変数 $\mathbfit{x}$ の次元数であり、$\boldsymbol{\mu}$ および $\boldsymbol{\Sigma}$ は以下に定義される平均ベクトル、および共分散行列である。<br>

$$
\boldsymbol{\mu} = (\mu^{(1)}, \mu^{(2)}, \cdots, \mu^{(M)})
$$

$$
\boldsymbol{\Sigma} = 
$$

#### 2. 異常度の定義

#### 3. 閾値の設定
